In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#ls gdrive/'My Drive'/raindrop_data/

In [ ]:
from shutil import copyfile
#copyfile('gdrive/My Drive/dl_proj_data/RAIN_DATASET_v5.zip', 'RAIN_DATASET_v5.zip')
#copyfile('gdrive/My Drive/raindrop_data/test_a.zip', 'test_a.zip')
#copyfile('gdrive/My Drive/raindrop_data/test_b.zip', 'test_b.zip')
#copyfile('gdrive/My Drive/raindrop_data/train.zip', 'train.zip')

In [ ]:
#!unzip RAIN_DATASET_v5.zip
#!unzip train.zip

In [ ]:
#ls train

In [ ]:
#ls RAIN_DATASET/ALIGNED_PAIRS/

In [ ]:
#mkdir data

In [ ]:
#ls data/CLEAN/

In [ ]:
#cp -R RAIN_DATASET/ALIGNED_PAIRS/CLEAN data/

In [ ]:
#cp -R RAIN_DATASET/ALIGNED_PAIRS/CG_DROPLETS data/

In [ ]:
import torch.nn as nn
import torch
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from torchvision import datasets,models,transforms
import matplotlib.pyplot as plt
import cv2
import random
from skimage.measure import compare_psnr, compare_ssim

In [ ]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class Reshape(nn.Module):
    def forward(self, input):
        return input.view(-1,1,input_shape[0],input_shape[1])
      
def conv_block(in_channels, out_channels,kernel=3,stride=2,padding=1):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels,kernel ,stride,padding=padding),
        nn.InstanceNorm2d(out_channels),
        nn.LeakyReLU(0.2,True)
    )
def conv_block_noNorm(in_channels, out_channels,kernel=3,stride=2,padding=1):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels,kernel ,stride,padding=padding),
        nn.LeakyReLU(0.2,True)
    )
def conv_block_noAct(in_channels, out_channels,kernel=3,stride=2,padding=1):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels,kernel ,stride,padding=padding),
        nn.InstanceNorm2d(out_channels),
    )
def conv_block2(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels,3,padding=1),
        nn.InstanceNorm2d(out_channels),
        nn.LeakyReLU(0.2,True)
    ) 
  

    def forward(self, input):
        return input.view(input.size(0), -1)
      
class resnet_block(nn.Module):
    def __init__(self,in_ch,out_ch):
        super(resnet_block, self).__init__()
        self.pad=nn.ReflectionPad2d(1)
        self.econv1=conv_block(in_ch,out_ch,stride=1,padding=0)
        self.econv2=conv_block_noAct(out_ch, out_ch,stride=1,padding=0)
    
    def forward(self, input):
      conv=self.pad(input)
      conv = self.econv1(conv)
      conv=self.pad(conv)
      conv=self.econv2(conv)
      return conv+input


      return input
'''
def custom_conv2d_padded(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 1, 1),
        nn.ReLU(inplace=True)
    )
'''
def custom_conv2dTranspose(in_channels, out_channels):
    return nn.Sequential(
        nn.ConvTranspose2d(in_channels, out_channels, 3,2,padding=1,output_padding=1),
        nn.InstanceNorm2d(out_channels),
        nn.LeakyReLU(0.2,True)
    )

In [ ]:
class pix2pix(nn.Module):
    def __init__(self, ngpu):
        super(pix2pix, self).__init__()
        self.ngpu = ngpu
        self.reflectpad3=nn.ReflectionPad2d(3)
        self.econv1=conv_block(3,32,kernel=7,stride=1,padding=0)
        self.econv2=conv_block(32,64)
        self.econv3=conv_block(64,128)
        self.econv4=conv_block(128,256)
        self.resnet1=resnet_block(256,256)
        self.resnet2=resnet_block(256,256)
        self.resnet3=resnet_block(256,256)
        self.resnet4=resnet_block(256,256)
        self.resnet5=resnet_block(256,256)
        self.resnet6=resnet_block(256,256)
        self.resnet7=resnet_block(256,256)
        self.resnet8=resnet_block(256,256)
        self.resnet9=resnet_block(256,256)
        #self.dconv1=conv_block2(32,3)
        #self.dconv2=conv_block2(64,32)
        #self.dconv3=conv_block2(128,64)
        #self.dconv4=conv_block2(256,128)
        self.upsample4=custom_conv2dTranspose(256,128)
        self.upsample3=custom_conv2dTranspose(256,64)
        self.upsample2=custom_conv2dTranspose(128,32)
        self.econv5=conv_block(64,3,kernel=7,stride=1,padding=0)
        #self.upsample1=custom_conv2dTranspose(64,3)
        #self.crop   = torch.nn.ReflectionPad2d((0,-15,0,-15))
        #self.output_layer1=custom_conv2d_padded(16*2**(0), 1)
        #self.output_layer2=custom_conv2d_padded(16*2**(0), 1)
    
    def forward(self, input):
      input=self.reflectpad3(input)
      #print('input ',input.size())
      conv1 = self.econv1(input)
      #print('econv1',conv1.size())

      conv2 = self.econv2(conv1)
      #print('econv2',conv2.size())

      conv3 = self.econv3(conv2)
      #print('econv3',conv3.size())

      conv4 = self.econv4(conv3)
      #print('econv4',conv4.size())
      resnet1=self.resnet1(conv4)
      resnet2=self.resnet2(resnet1)
      resnet3=self.resnet3(resnet2)
      resnet4=self.resnet4(resnet3)
      resnet5=self.resnet5(resnet4)
      resnet6=self.resnet6(resnet5)
      resnet7=self.resnet7(resnet6)
      resnet8=self.resnet8(resnet7)
      resnet9=self.resnet9(resnet8) 
      #print('resnet',resnet1.size())
      input = self.upsample4(resnet9)
      #print('upsample4',input.size())
      input= torch.cat([input, conv3], dim=1)
      #print('cat',input.size())
      #input=self.dconv4(input)
      #print('dconv4',input.size())
      input= self.upsample3(input)   
      #print('upsample3',input.size())
      input = torch.cat([input, conv2], dim=1) 
      #print('cat',input.size())
      #input=self.dconv3(input)
      #print('dconv3',input.size())

      input = self.upsample2(input)  
      #print('upsample2',input.size())

      input = torch.cat([input, conv1], dim=1)
      #print(input.size())
      #input=self.dconv2(input)
      #print('dconv2',input.size())
      input=self.reflectpad3(input)
      input = self.econv5(input)  
      #print('econv5',input.size())
      #input = torch.cat([input, conv1], dim=1)
      #print(input.size())
      #print('output size',input.size())
      #print('output size',input.size())
      return input

In [ ]:
ngpu=1
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
netG = pix2pix(ngpu).to(device)

In [ ]:
#temp=torch.randn(4,3,320,320 ,device=device)
#a=netG(temp)

In [ ]:

class patchgan_disc(nn.Module):
    def __init__(self, ngpu):
      super(patchgan_disc, self).__init__()
      self.ngpu = ngpu
      self.conv1=conv_block_noNorm(3,64,kernel=3,stride=2,padding=0)
      self.conv2=conv_block(64,128,kernel=3,stride=2,padding=0)
      self.conv3=conv_block(128,256,kernel=3,stride=2,padding=0)
      self.conv4=conv_block(256,512,kernel=3,stride=2,padding=0)
      self.conv5=nn.Conv2d(512, 1,3,stride=2,padding=0)
    
    def forward(self, input):
      input=self.conv1(input)
      input=self.conv2(input)
      input=self.conv3(input)
      input=self.conv4(input)
      input=self.conv5(input)

      return input
'''
class patchgan_disc(nn.Module):
    def __init__(self, ngpu):
      super(patchgan_disc, self).__init__()
      self.ngpu = ngpu
      self.conv1=conv_block_noNorm(3,64,kernel=4,stride=2,padding=0)
      self.conv2=conv_block(64,128,kernel=4,stride=2,padding=0)
      self.conv3=conv_block(128,256,kernel=4,stride=2,padding=0)
      self.conv4=conv_block(256,512,kernel=4,stride=1,padding=0)
      self.conv5=conv_block(512,1,kernel=4,stride=1,padding=0)
      self.flatten=Flatten()
      self.linear1=nn.Linear(2704,1024)
      self.act1=nn.LeakyReLU(0.2,True)
      self.linear2=nn.Linear(1024, 1)
      self.act2=nn.Sigmoid()
    def forward(self, input):
      input=self.conv1(input)
      input=self.conv2(input)
      input=self.conv3(input)
      input=self.conv4(input)
      input=self.conv5(input)
      input=self.flatten(input)
      input=self.linear1(input)
      input=self.act1(input)
      input=self.linear2(input)
      input=self.act2(input)
      return input
'''

'\nclass patchgan_disc(nn.Module):\n    def __init__(self, ngpu):\n      super(patchgan_disc, self).__init__()\n      self.ngpu = ngpu\n      self.conv1=conv_block_noNorm(3,64,kernel=4,stride=2,padding=0)\n      self.conv2=conv_block(64,128,kernel=4,stride=2,padding=0)\n      self.conv3=conv_block(128,256,kernel=4,stride=2,padding=0)\n      self.conv4=conv_block(256,512,kernel=4,stride=1,padding=0)\n      self.conv5=conv_block(512,1,kernel=4,stride=1,padding=0)\n      self.flatten=Flatten()\n      self.linear1=nn.Linear(2704,1024)\n      self.act1=nn.LeakyReLU(0.2,True)\n      self.linear2=nn.Linear(1024, 1)\n      self.act2=nn.Sigmoid()\n    def forward(self, input):\n      input=self.conv1(input)\n      input=self.conv2(input)\n      input=self.conv3(input)\n      input=self.conv4(input)\n      input=self.conv5(input)\n      input=self.flatten(input)\n      input=self.linear1(input)\n      input=self.act1(input)\n      input=self.linear2(input)\n      input=self.act2(input)\n      re

In [ ]:
netD=patchgan_disc(ngpu).to(device)

In [ ]:
#temp=torch.randn(4,3,320,320 ,device=device)
#a=netD(temp)
#b=a.view(-1).size()
#print(list(b)[0])
#asd

In [ ]:
class disc_loss_func(nn.Module):
    def __init__(self,ngpu):
        super(disc_loss_func, self).__init__()
    
    def forward(self,rainy,clean,netD):
      clean=netD(clean)
      #gen_label=torch.tensor(0.0).expand_as(rainy).to(device)
      clean_label=torch.tensor(1.0).expand_as(clean).to(device)
      loss=(( rainy**2) + ((clean-clean_label)**2)).mean()
      return loss
  
class gen_loss_func(nn.Module):
    def __init__(self,ngpu,vgg_model):
        super(gen_loss_func, self).__init__()
        #self.vgg_layers = vgg_model.features
        self.vgg_model=vgg_model
        #self.netD=netD.features
    
    def forward(self,rainy,clean,netG,netD): #rainy is D (G (rainy )) 
      generated=netG(rainy)
      
      Le=(generated-clean)**2
      div=list(generated.view(-1).size())[0]
      Le=Le.sum()/div
      #Le=Le.mean()
      
      rainy=netD(generated)
      real_label=torch.tensor(1.0).expand_as(rainy).to(device)
      adv_loss= ((rainy-real_label)**2).mean()
      nvgg=31#50#31
      i=0
      Lperc=0
      scale=1e6
      temp_generated=generated
      temp_clean=clean
      #temp_generated=temp_generated*255.0
      #temp_clean=temp_clean*255.0
      '''
      for name, module in self.vgg_layers._modules.items():
            w=2**(nvgg-i)
            i=i+1
            temp_generated = module(temp_generated)
            div=list(temp_generated.view(-1).size())[0]
            temp_clean = module(temp_clean)
      '''
      temp_generated=self.vgg_model(generated)
      temp_clean=self.vgg_model(temp_clean)
      temp=((temp_generated-temp_clean)**2).sum()
      #print(temp)
      #temp=temp/scale
      temp=temp/div
      Lperc=Lperc+temp
      '''     
      nlayers=15#28#5
      i=0
      Lmsadv=0
      temp_generated=generated
      temp_clean=clean
      #temp_generated=temp_generated*255.0
      #temp_clean=temp_clean*255.0
      for module in netD.children():
            w=2**(nlayers-i)
            i=i+1
            temp_generated = module(temp_generated)
            div=list(temp_generated.view(-1).size())[0]
            temp_clean = module(temp_clean)
            temp=torch.abs(temp_generated-temp_clean).sum()
            #temp=temp/scale
            temp=temp/(w*div)
            Lmsadv=Lmsadv+temp
      '''
      #print(0.001*adv_loss.item(),Le.item(),Lperc.item())#,Lmsadv.item())
      loss=0.01*adv_loss+Lperc+Le#+Lmsadv
      return loss

In [ ]:
#f#or module in netD.children():
  #print(module)
  #asd

In [ ]:
epochs=100000
batch=1
lr_gen=0.0001
lr_disc=0.0001
display=500
def trainable(net, trainable):
    for para in net.parameters():
        para.requires_grad = trainable
vgg_model = models.vgg16(pretrained=True)
vgg_model = torch.nn.Sequential(*(list(vgg_model.children())[:-2]))
trainable(vgg_model, False)
disc_loss=disc_loss_func(ngpu).to(device)
gen_loss=gen_loss_func(ngpu,vgg_model).to(device)
optimizer_G = torch.optim.Adam(netG.parameters(), lr=lr_gen)
optimizer_D = torch.optim.Adam(netD.parameters(), lr=lr_disc)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:04<00:00, 137265522.83it/s]


In [ ]:
'''
data_transform = transforms.Compose([
        transforms.Resize((640,640)),
        transforms.ToTensor()
    ])
dataset = datasets.ImageFolder(root='data/',
                                           transform=data_transform)
dataset_loader = torch.utils.data.DataLoader(dataset,
                                             batch_size=batch, shuffle=True,
                                             num_workers=0)
data_iter=iter(dataset_loader)
'''

"\ndata_transform = transforms.Compose([\n        transforms.Resize((640,640)),\n        transforms.ToTensor()\n    ])\ndataset = datasets.ImageFolder(root='data/',\n                                           transform=data_transform)\ndataset_loader = torch.utils.data.DataLoader(dataset,\n                                             batch_size=batch, shuffle=True,\n                                             num_workers=0)\ndata_iter=iter(dataset_loader)\n"

In [ ]:

class data_load():
    def __init__(self,path_to_clean='data/',batch=5):
        super(data_load, self).__init__()
        self.clean_files=glob(path_to_clean+'CLEAN/'+'*')
        self.path_to_clean=path_to_clean
        self.batch=batch
        
    def next(self):
        size=480
        clean_imgs=np.zeros((self.batch,3,size,size))
        rainy_imgs=np.zeros((self.batch,3,size,size))
        for i in range(self.batch):
          flip_rand=random.randint(0,2)
          index=random.randint(0,len(self.clean_files)-1)
          single_clean=self.clean_files[index][15:]
          rainy_path=self.path_to_clean+'CG_DROPLETS/left'+single_clean
          rainy_img=cv2.imread(rainy_path)
          x_rand=random.randint(0,rainy_img.shape[0]-size)
          y_rand=random.randint(0,rainy_img.shape[1]-size)
          rainy_img=rainy_img[x_rand:x_rand+size,y_rand:y_rand+size,:]
          if flip_rand>1:
            rainy_img=cv2.flip(rainy_img, 1 )
          #rainy_img=cv2.resize(rainy_img,(256,256))
          rainy_img=np.transpose(rainy_img, (2, 0, 1))
          clean_img=cv2.imread(self.clean_files[index])
          clean_img=clean_img[x_rand:x_rand+size,y_rand:y_rand+size,:]
          #clean_img[clean_img<0]=0
          if flip_rand>1:
            clean_img=cv2.flip(clean_img, 1 )
          #clean_img=cv2.resize(clean_img,(256,256))
          clean_img=np.transpose(clean_img, (2, 0, 1))
          clean_img=clean_img/255
          rainy_img=rainy_img/255
          clean_imgs[i]=clean_img
          rainy_imgs[i]=rainy_img
        return rainy_imgs,clean_imgs
      
''' 
class data_load():
    def __init__(self,path_to_clean='train/',batch=5):
        super(data_load, self).__init__()
        self.clean_files=glob(path_to_clean+'gt/*')
        self.path_to_clean=path_to_clean
        self.batch=batch
        
    def next(self):
        size=480
        clean_imgs=np.zeros((self.batch,3,size,size))
        rainy_imgs=np.zeros((self.batch,3,size,size))
        for i in range(self.batch):
          index=random.randint(0,len(self.clean_files)-1)
          flip_rand=random.randint(0,2)
          single_clean=self.clean_files[index][9:]
          single_clean=single_clean.split('_')[0]
          rainy_path=self.path_to_clean+'data/'+single_clean+'_rain.png'
          rainy_img=cv2.imread(rainy_path)
          x_rand=random.randint(0,rainy_img.shape[0]-size)
          y_rand=random.randint(0,rainy_img.shape[1]-size)
          rainy_img=rainy_img[x_rand:x_rand+size,y_rand:y_rand+size,:]
          #rainy_img=cv2.resize(rainy_img,(480,480))
          #rainy_img=rainy_img-int(scale_rand)
          #rainy_img[rainy_img<0]=0
          if flip_rand>1:
            rainy_img=cv2.flip(rainy_img, 1 )
            #plt.imshow(rainy_img)
            #plt.show()
          rainy_img=np.transpose(rainy_img, (2, 0, 1))
          clean_img=cv2.imread(self.clean_files[index])
          #clean_img=cv2.resize(clean_img,(480,480))
          clean_img=clean_img[x_rand:x_rand+size,y_rand:y_rand+size,:]
          #clean_img[clean_img<0]=0
          if flip_rand>1:
            clean_img=cv2.flip(clean_img, 1 )
          clean_img=np.transpose(clean_img, (2, 0, 1))
          clean_img=clean_img/255
          rainy_img=rainy_img/255
          clean_imgs[i]=clean_img
          rainy_imgs[i]=rainy_img
        return rainy_imgs,clean_imgs
'''

" \nclass data_load():\n    def __init__(self,path_to_clean='train/',batch=5):\n        super(data_load, self).__init__()\n        self.clean_files=glob(path_to_clean+'gt/*')\n        self.path_to_clean=path_to_clean\n        self.batch=batch\n        \n    def next(self):\n        size=480\n        clean_imgs=np.zeros((self.batch,3,size,size))\n        rainy_imgs=np.zeros((self.batch,3,size,size))\n        for i in range(self.batch):\n          index=random.randint(0,len(self.clean_files)-1)\n          flip_rand=random.randint(0,2)\n          single_clean=self.clean_files[index][9:]\n          single_clean=single_clean.split('_')[0]\n          rainy_path=self.path_to_clean+'data/'+single_clean+'_rain.png'\n          rainy_img=cv2.imread(rainy_path)\n          x_rand=random.randint(0,rainy_img.shape[0]-size)\n          y_rand=random.randint(0,rainy_img.shape[1]-size)\n          rainy_img=rainy_img[x_rand:x_rand+size,y_rand:y_rand+size,:]\n          #rainy_img=cv2.resize(rainy_img,(480,

In [ ]:
data_loader=data_load(batch=batch)
netG=torch.load('gdrive/My Drive/netG.pt')
netD=torch.load('gdrive/My Drive/netD.pt')

In [ ]:
def test(rainy,netG):
  index=0
  generated=netG(rainy)[index]
  rainy=rainy[index]
  generated=generated.cpu().detach().numpy()
  img=rainy.cpu().detach().numpy()
  img=np.transpose(img, (1, 2, 0))
  generated=np.transpose(generated, (1, 2, 0))
  generated[generated<0]=0.0
  generated[generated>1]=1.0
  plt.imshow(img)
  plt.title('rainy')
  plt.show()
  plt.imshow(generated)
  plt.title('de rained')
  plt.show()

In [ ]:
for epoch in range(epochs):
  netD.zero_grad()
  for param in netD.parameters():
     param.requires_grad = True
  rainy,clean=data_loader.next()
  rainy=torch.from_numpy(rainy).to(device).float()
  
  clean=torch.from_numpy(clean).to(device).float()
  generated=netG(rainy).to(device)
  D_rainy=netD(generated.detach())
  loss=disc_loss(D_rainy,clean,netD)
  loss.backward()
  optimizer_D.step()
  netG.zero_grad()
  #netD.zero_grad()
  for param in netD.parameters():
     param.requires_grad = False
  loss2=gen_loss(rainy,clean,netG,netD)
  loss2.backward()
  optimizer_G.step()
  if epoch%display==0:
    test(rainy,netG)
    print('epoch:',epoch,'Discriminator loss:',loss.item(),'generator loss:',loss2.item())

ValueError: ignored

In [ ]:
torch.save(netG, 'gdrive/My Drive/netG.pt')
torch.save(netD, 'gdrive/My Drive/netD.pt')

In [ ]:
netG=torch.load('gdrive/My Drive/final weights/Copy of netG.pt')

In [ ]:
from google.colab import files
!rm case1.jpg
files.upload()
rainy=plt.imread('case1.jpg')
if np.max(rainy)>1.0:
  rainy=rainy/255

if rainy.shape[0]<rainy.shape[1]:
  size=int(rainy.shape[0]/32)*32
else:
  size=int(rainy.shape[1]/32)*32
rainy=cv2.resize(rainy,(size,size))
rainy=np.transpose(rainy,[2,0,1])
rainy=np.expand_dims(rainy,axis=0)
rainy=torch.from_numpy(rainy).to(device).float()
test(rainy,netG)

# PSNR calculation

In [ ]:
netG=torch.load('gdrive/My Drive/final weights/Copy of netG.pt')

In [ ]:
!ls gdrive/'My Drive'/raindrop_data/

test_a.zip  test_b.zip	train.zip


In [ ]:
from shutil import copyfile
#copyfile('gdrive/My Drive/raindrop_data/test_a.zip', 'test_a.zip')

'test_a.zip'

In [ ]:
#!unzip test_a.zip

In [ ]:
#ls test_a/data

In [ ]:
clean_test_images_files=glob('test_a/gt/*.png')
rain_test_images_files=glob('test_a/data/*.png')

In [ ]:
rain_test_images_files[:2]

['test_a/data/5_rain.png', 'test_a/data/44_rain.png']

In [ ]:
def test_metric(rainy,netG):
  index=0
  generated=netG(rainy)[index]
  rainy=rainy[index]
  generated=generated.cpu().detach().numpy()
  img=rainy.cpu().detach().numpy()
  img=np.transpose(img, (1, 2, 0))
  generated=np.transpose(generated, (1, 2, 0))
  generated[generated<0]=0.0
  generated[generated>1]=1.0

  return generated

In [ ]:
def calc_psnr(im1, im2):
    im1_y = cv2.cvtColor(im1, cv2.COLOR_BGR2YCR_CB)[:, :, 0]
    im2_y = cv2.cvtColor(im2, cv2.COLOR_BGR2YCR_CB)[:, :, 0]
    return compare_psnr(im1_y, im2_y)


def calc_ssim(im1, im2):
    im1_y = cv2.cvtColor(im1, cv2.COLOR_BGR2YCR_CB)[:, :, 0]
    im2_y = cv2.cvtColor(im2, cv2.COLOR_BGR2YCR_CB)[:, :, 0]
    return compare_ssim(im1_y, im2_y)

In [ ]:
psnr=[]
ssim=[]
for i in range(len(clean_test_images_files)):
  path_to_clean='test_a/gt/'+rain_test_images_files[i][12:].split('_')[0]+'_clean.png'
  rainy=plt.imread(rain_test_images_files[i])
  if np.max(rainy)>1.0:
    rainy=rainy/255

  if rainy.shape[0]<rainy.shape[1]:
    size=int(rainy.shape[0]/32)*32
  else:
    size=int(rainy.shape[1]/32)*32
  rainy=cv2.resize(rainy,(size,size))
  plt.imshow(rainy)
  plt.title('rainy')
  plt.show()
  rainy=np.transpose(rainy,[2,0,1])
  rainy=np.expand_dims(rainy,axis=0)
  rainy=torch.from_numpy(rainy).to(device).float()
  generated=test_metric(rainy,netG)
  plt.imshow(generated)
  plt.title('generated')
  plt.show()
  clean=plt.imread(path_to_clean)
  if np.max(clean)>1.0:
    clean=clean/255

  if clean.shape[0]<clean.shape[1]:
    size=int(clean.shape[0]/32)*32
  else:
    size=int(clean.shape[1]/32)*32
  clean=cv2.resize(clean,(size,size))
 
  plt.imshow(clean)
  plt.title('clean')
  plt.show()
  psnr.append(calc_psnr(generated,clean))
  ssim.append(calc_ssim(generated,clean))

  

In [ ]:
np.mean(psnr)

19.35939851637655

In [ ]:
np.mean(ssim)

0.752125042093159

In [ ]:
rainy=plt.imread(rain_test_images_files[0])
if np.max(rainy)>1.0:
  rainy=rainy/255

if rainy.shape[0]<rainy.shape[1]:
  size=int(rainy.shape[0]/32)*32
else:
  size=int(rainy.shape[1]/32)*32
rainy=cv2.resize(rainy,(size,size))
print(rainy.shape)
rainy=np.transpose(rainy,[2,0,1])
rainy=np.expand_dims(rainy,axis=0)
rainy=torch.from_numpy(rainy).to(device).float()
generated=test_metric(rainy,netG)
print(generated.shape)

(512, 512, 3)
(512, 512, 3)
